<a href="https://colab.research.google.com/github/Eezzeldin/candy/blob/main/candy_EDA_4_multioutput_counterfactuals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
import numpy as np
from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.multioutput import RegressorChain
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor, MultiOutputClassifier
from sklearn.preprocessing import FunctionTransformer
from itertools import product
import numpy as np


In [ ]:
#!pip install --upgrade scikit-learn

In [ ]:
# Load the dataset
df = pd.read_csv('/content/candy-data.csv')

# Display the first few rows of the dataset
df.head()

,competitorname,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent
0,100 Grand,1,0,1,0,0,1,0,1,0,0.732,0.860,66.971725
1,3 Musketeers,1,0,0,0,1,0,0,1,0,0.604,0.511,67.602936
2,One dime,0,0,0,0,0,0,0,0,0,0.011,0.116,32.261086
3,One quarter,0,0,0,0,0,0,0,0,0,0.011,0.511,46.116505
4,Air Heads,0,1,0,0,0,0,0,0,0,0.906,0.511,52.341465


In [ ]:
df.columns

Index(['competitorname', 'chocolate', 'fruity', 'caramel', 'peanutyalmondy',
       'nougat', 'crispedricewafer', 'hard', 'bar', 'pluribus', 'sugarpercent',
       'pricepercent', 'winpercent'],
      dtype='object')

In [ ]:
df ["winpercent"] = df ["winpercent"] / 100

In [ ]:
#Stage 1 : Fit RandomForestRegressor on real data in a multioutput regressor predicting the 3 continouss variables.
#Stage 2 : Function transformer generating all possible combinations from binary variables.
#Stage 3 : Function Transformer  of the regressor in Stage 1 predicting on all possible combination
#Stage 4 : Function Transformer of appending real to counterfactual data and putting a flag indicator.
#Stage 5 : Multioutput RandomForestClassifier training on continous variables and predicting on binary ones.

In [ ]:
'regressor' : RandomForestRegressor ()


def generate_counterfactuals (X) :
    return counterfactuals




In [ ]:
def train_on_real_data (X) :
    binary_features = X.iloc[:, :-3]
    targets    =  X.iloc[:, -3:]
    regressor = MultiOutputRegressor(RandomForestRegressor())
    return  regressor.fit (binary_features , targets)


# Function to generate all possible combinations of binary variables
def generate_combinations(regressor , X  ):
    # Assuming binary features are all except the last three continuous features
    binary_features = X[:, :-3]
    combinations = np.array(list(product([0, 1], repeat=binary_features.shape[1])))
    return combinations

# Function to predict with the regressor on combinations
def predict_combinations(regressor , X):
    combinations = generate_combinations(X)
    predictions = regressor.predict(combinations)
    return predictions

# Function to append real data to counterfactual predictions
def append_real_counterfactual_data(X, predictions):
    # Adding flag to indicate real (1) or counterfactual (0) data
    real_data = np.hstack((X, np.ones((X.shape[0], 1))))
    counterfactual_data = np.hstack((predictions, np.zeros((predictions.shape[0], 1))))
    return np.vstack((real_data, counterfactual_data))

def classifier_predict_binary (X) :
    classifier = MultiOutputClassifier(RandomForestClassifier())
    features   = X [ ["winpercent" , "sugarpercent" , "pricepercent"] ]
    targets    = X.drop (["winpercent" , "sugarpercent" , "pricepercent"] , axis = 1)
    return classifier.fit (features , targets)




# Creating the pipeline stages
regressor             = FunctionTransformer (train_on_real_data)
generate_combinations = FunctionTransformer (generate_combinations ,  kw_args={'regressor': regressor } )
predictor_transformer = FunctionTransformer(predict_combinations, kw_args={'regressor': regressor})
append_transformer    = FunctionTransformer(append_real_counterfactual_data)
classifier            = FunctionTransformer (classifier_predict_binary)

# Pipeline
pipeline = Pipeline(steps=[
    ('regressor', regressor),
    ('generate_combinations',generate_combinations) ,
    ('predictor_transformer', predictor_transformer),
    ('append_transformer', append_transformer),
    ('classifier', classifier)
])


pipeline = Pipeline(steps=[
    ('regressor', regressor),
    ('generate_combinations',generate_combinations)

])

# Show the pipeline
pipeline


Pipeline(steps=[('regressor',
                 FunctionTransformer(func=<function train_on_real_data at 0x7c3a95836200>)),
                ('generate_combinations',
                 FunctionTransformer(func=<function generate_combinations at 0x7c3a95834310>,
                                     kw_args={'regressor': FunctionTransformer(func=<function train_on_real_data at 0x7c3a95836200>)}))])

In [ ]:
pipeline.transform (df.drop ("competitorname" ,axis = 1))

TypeError: generate_combinations() got multiple values for argument 'regressor'

In [ ]:
pipeline.transform (df.drop ("competitorname" ,axis = 1))

TypeError: generate_combinations() got multiple values for argument 'regressor'

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
'''
# Custom transformer for handling the regressor
class CustomRegressor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = MultiOutputRegressor(RandomForestRegressor())

    def fit(self, X, y):
        binary_features = X  # Assuming last three columns are targets
        self.model.fit(binary_features, y)
        return self

    def transform(self, X):
        # This could return the model's predictions or some other transformation
        return self.model.predict(X)
'''
# Custom transformer to generate and predict combinations
class CombinationsPredictor(BaseEstimator, TransformerMixin):
    def __init__(self, regressor):
        self.regressor = regressor

    def fit(self, X, y=None):
        binary_features = X  # Assuming last three columns are targets
        self.regressor.fit(binary_features, y)
        return self

    def transform(self, X):
        combinations = np.array(list(product([0, 1], repeat=X.shape[1])))
        predictions  = self.regressor.predict(combinations)

        real_data           = np.hstack((X, np.ones((X.shape[0], 1)) , y ))
        counterfactual_data = np.hstack((combinations, np.zeros((predictions.shape[0], 1)) , predictions))

        return np.vstack((real_data, counterfactual_data))


# Custom transformer to generate and predict combinations
class BinaryClassifier(BaseEstimator, TransformerMixin):
    def __init__(self, classifier):
        self.classifier = classifier

    def fit(self, X, y=None):
        print (y)
        print (X)
        binary_features = X [:,:-4]  # Assuming last three columns are targets
        cont_targets    = X [:,-4:]
        self.classifier.fit ( cont_targets , binary_features  )
        return self

    def transform(self, X):
        cont_targets    = X [:,-4:]
        predictions     = self.classifier.predict(cont_targets )
        return predictions





# Pipeline
pipeline = Pipeline(steps=[
    ('combinations_predictor', CombinationsPredictor(  MultiOutputRegressor( RandomForestRegressor() ) ) ),
    ('binary_classifier', BinaryClassifier (  MultiOutputClassifier( RandomForestClassifier() ) ) )
    # Add more steps as necessary
])

# Note: Actual usage might need further adjustments based on your specific requirements
pipeline

Pipeline(steps=[('combinations_predictor',
                 CombinationsPredictor(regressor=MultiOutputRegressor(estimator=RandomForestRegressor()))),
                ('binary_classifier',
                 BinaryClassifier(classifier=MultiOutputClassifier(estimator=RandomForestClassifier())))])

In [ ]:
X  = df.iloc[:, :-3].drop ("competitorname",axis =1)
X.head ()

,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus
0,1,0,1,0,0,1,0,1,0
1,1,0,0,0,1,0,0,1,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0


In [ ]:
y = df.iloc[:, -3:]
y.head ()

,sugarpercent,pricepercent,winpercent
0,0.732,0.860,0.669717
1,0.604,0.511,0.676029
2,0.011,0.116,0.322611
3,0.011,0.511,0.461165
4,0.906,0.511,0.523415


In [ ]:
pipeline [0]

CombinationsPredictor(regressor=MultiOutputRegressor(estimator=RandomForestRegressor()))

In [ ]:
pipeline [1]

BinaryClassifier(classifier=MultiOutputClassifier(estimator=RandomForestClassifier()))

In [ ]:
pipeline.fit (X, y)

Pipeline(steps=[('combinations_predictor',
                 CombinationsPredictor(regressor=MultiOutputRegressor(estimator=RandomForestRegressor())))])

In [ ]:
pd.DataFrame ( pipeline [0].fit_transform (X.values, y) )

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.732000,0.860000,0.669717
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.604000,0.511000,0.676029
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.011000,0.116000,0.322611
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.011000,0.511000,0.461165
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.906000,0.511000,0.523415
...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.658217,0.679252,0.655133
593,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.466135,0.616711,0.728663
594,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.506612,0.671728,0.686849
595,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.447889,0.666629,0.653915


In [ ]:
pd.DataFrame ( pipeline.fit_transform (X.values, y) )

    sugarpercent  pricepercent  winpercent
0          0.732         0.860    0.669717
1          0.604         0.511    0.676029
2          0.011         0.116    0.322611
3          0.011         0.511    0.461165
4          0.906         0.511    0.523415
..           ...           ...         ...
80         0.220         0.116    0.454663
81         0.093         0.116    0.390119
82         0.313         0.313    0.443755
83         0.186         0.267    0.419043
84         0.872         0.848    0.495241

[85 rows x 3 columns]
[[1.         0.         1.         ... 0.73199999 0.86000001 0.66971725]
 [1.         0.         0.         ... 0.60399997 0.51099998 0.67602936]
 [0.         0.         0.         ... 0.011      0.116      0.32261086]
 ...
 [1.         1.         1.         ... 0.53744151 0.67077824 0.67875912]
 [1.         1.         1.         ... 0.45186932 0.66853211 0.65676506]
 [1.         1.         1.         ... 0.52233604 0.67464989 0.64954222]]


,0,1,2,3,4,5,6,7,8
0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
592,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
593,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
594,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
595,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
